### Train BERT from Scratch using Transformers in Python
* https://www.thepythoncode.com/article/pretraining-bert-huggingface-transformers-in-python

In [1]:
# !pip install datasets transformers==4.18.0 sentencepiece

In [1]:
import os
import json
import re
import unicodedata
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from transformers import *
from tokenizers import *
from datasets import *
from sklearn.model_selection import train_test_split
import nltk
from nltk.data import load

/home/hdh/PycharmProjects/bert_pretrain/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = "c09k_pretrained_bert"
### Tokenizer Hyperparam
# 30,522 vocab is BERT's default vocab size, feel free to tweak
vocab_size = 8000
# maximum sequence length, lowering will result to faster training (when increasing batch size)
max_length = 512
# whether to truncate
truncate_longer_samples = False

In [3]:
# 다운로드한 원시 데이터
kiwee_files = ['data/c09k_0001-1000.xlsx', 'data/c09k_1001-2000.xlsx', 'data/c09k_2001-3000.xlsx', 'data/c09k_3001-3935.xlsx']
# 문장 분리의 정확도를 높이기 위해 유니코드 문자(JP 번역), 약어 이후의 마침표는 문장 구분시 제외 처리하도록
# NLTK의 tokenizer의 abbrev_types을 업데이트 하여 sent_tokenizer 객체 생성, sent_tokenizer 적용한 텍스트 생성
# tokenizer 학습할 때 한문은 vocab에 추가되지 않도록 예외처리
corpus_after_cleansing = 'data/c09k_corpus.txt'

### Utility 함수

In [4]:
# 아래 링크를 참조하여 약어 이후에 등장하는 마침표를 사용해 문장이 분리되지 않도록 NLTK의 tokenizer를 사용해 문장 분리
# https://cryptosalamander.tistory.com/140?category=1218889
sent_tokenizer = load("tokenizers/punkt/english.pickle")
extra_abbreviations = [
    'RE','re','pat', 'no', 'nos','vol','jan','feb','mar','apr','jun',
    'jul','aug','sep','oct','nov','dec','eng','ser','ind','ed','pp',
    'e.g','al','T.E.N.S', 'E.M.S','F.E','U.H.T.S.T','degree',
    '/gm','A','B','C','D','E','F','G','H','I','J','K','L','M','N','O',
    'P','Q','R','S','T','U','V','W','X','Y','Z']
sent_tokenizer._params.abbrev_types.update(extra_abbreviations)
# NLTK의 tokenizer를 사용해 문장 분리(미사용)
# https://cryptosalamander.tistory.com/140?category=1218889

In [5]:
def cleansing_corpus():
    try:
        dataset = Dataset.from_text(corpus_after_cleansing)
        print('Loading from ', corpus_after_cleansing, ' completed')
    except:
        kiwee_files = ['data/c09k_0001-1000.xlsx', 'data/c09k_1001-2000.xlsx', 'data/c09k_2001-3000.xlsx', 'data/c09k_3001-3935.xlsx']
        with open(corpus_after_cleansing, 'a') as f:
            f.truncate(0)
            for i, fn in enumerate(kiwee_files):
                tmp = pd.read_excel(fn).fillna('')
                # pandas는 비어있는 컬럼의 dtype을 float로 바꿔서 인식한다. 그로 인해 토크나이징 할 데이터가 없으면 오류가 발생되어 fillna를 사용해 모두 텍스트로 인식시키도록 한다
                # https://stackoverflow.com/questions/53953286/pandas-read-excel-blanks-in-string-columns-convert-to-floats-converting-via-st
                col_text = ['발행번호', '발명의명칭', '요약', '대표청구항', '과제', '해결방안']
                tmp = tmp[col_text]
                for index, row in tmp.iterrows():
            #         print(index, '\n', row['발명의명칭'], row['요약'], row['대표청구항'], row['과제'], row['해결방안'], '\n')
                    for col in col_text[1:]:
            #             print('처리중인 데이터:', col, row[col], '\n')
                        if row[col].strip() == "":
                            pass
                        else:
            #                 print(row[col].strip())
                            row[col] = unicodedata.normalize('NFKC', row[col])
                            # row[col] = unicodedata.normalize('NFC', row[col])  # 자음과 모음이 깨질 때는 NFC로 변환
                            # NFD(Normalization Form Decomposition) : 자음과 모음이 분리
                            # row[col] = unicodedata.normalize('NFKD', row[col])
                            #     https://blog.naver.com/PostView.nhn?blogId=duswl0319&logNo=221516880642&from=search&redirect=Log&widgetTypeCall=true&directAccess=false
                            row[col] = row[col].replace('\n\t',' ')
                            row[col] = row[col].replace('\n',' ')
                            row[col] = row[col].replace('&lt;',' ')
                            row[col] = row[col].replace('_x000d_',' ')
                            row[col] = row[col].replace('\t\t',' ')
                            row[col] = row[col].replace('@@',' ')
                            row[col] = row[col].replace('.  .','.')
                            row[col] = row[col].replace('. .','.')
                            row[col] = row[col].replace('..','.')
                            row[col] = row[col].replace('〜','~')
                            row[col] = row[col].replace(' . ','.')
                            row[col] = row[col].replace(' ． ','.')
                            row[col] = row[col].replace('． ','.')
                            row[col] = row[col].replace('. ','.')
                            row[col] = row[col].replace('  ',' ')
                            row[col] = row[col].replace('  ',' ')
                            row[col] = row[col].replace('【과제】',' ')
                            row[col] = row[col].replace('【요약】',' ')
                            row[col] = row[col].replace('【해결 수단】',' ')
                            str_tmp = sent_tokenizer.tokenize(row[col].strip())
            #                 print('문장 분리: ', str_tmp, '\n'*3)
            #                 result  = [f"{line}\n" for line in str_tmp]
                            for line in str_tmp:
                                f.write(f"{line}\n")
        dataset = Dataset.from_text(corpus_after_cleansing)
        print('Cleansing and savng to ', corpus_after_cleansing, ' completed')
    return dataset

In [6]:
def split_dataset():
    d = dataset.train_test_split(test_size=0.1)
    d["train"], d["test"]
    for t in d["train"]["text"][:3]:
        print(t)
        print("="*50)
    return d["train"], d["test"]

In [7]:
# if you want to train the tokenizer from scratch (especially if you have custom
# dataset loaded as datasets object), then run this cell to save it as files
# but if you already have your custom data as text files, there is no point using this
def dataset_to_text(dataset, output_filename="data.txt"):
    """Utility function to save dataset text to disk,
    useful for using the texts to train the tokenizer 
    (as the tokenizer accepts files)"""
    with open(output_filename, "w") as f:
        for t in dataset["text"]:
            print(t, file=f)

### Tokenizer train data 생성

In [40]:
# After cleansing and saving to 
dataset = cleansing_corpus()
print(dataset['text'][0:5])
print(dataset['text'][-5:])

Loading from c09k_corpus.txt completed
['이산화티탄-염화리튬 전기변색물질, 이를 이용한전기변색장치용 파우더 및 필름전극의 제조방법', '본 발명은 이산화티탄-염화리튬 전기변색물질 및 이를 이용한 전기변색장치용 파우더 및 필름전극의 제조방법에 관한 것으로, 이산화티탄에 염화리튬이 균일하게 혼합된 물질을 합성하여 제조되며 상기 이산화티탄-염화리튬에서 티탄 대 리튬의 몰 함량 비율이 1 : x (0.5 x 2) 범위를 갖는 이산화티탄-염화리튬을 졸-겔 방법으로 합성함으로써 표면적이 극대화되는 다공성화 또는 그 물질 내에 존재하는 높은 함량의 리튬 이온으로 인하여 전기변색 감응시간을 획기적으로 줄이고 전기변색 효율을 크게 향상시킬 수 있는 매우 뛰어난 발명인 것이다.', '이산화티탄-염화리튬 파우더에 아세틸아세톤이 포함된 증류수를 첨가하여 끈적거리는 상태가 될 때까지 교반하는 단계; 상기 용액에 증류수와 트리톤 X-100을 첨가하여 완전히 균일한 용액이 될 때까지 다시 교반하는 단계; 및 상기 용액을 전도성 유리인 ITO 위에 얇게 입힌 후, 상온에서 건조한 다음 열처리하는 단계를 포함함을 특징으로 하는 전기변색장치용 이산화티탄-염화리튬 필름전극의 제조방법.', '1 발명은 상기의 제문제점을 감안하여 안출한 것으로, 본 발명에서는 이산화티탄-염화리튬을 새롭게 합성하여 전기변색 물질로서 실용화함에 그 목적이 있다.', '2 발명의 다른 목적은 전기변색 감응시간을 획기적으로 줄이고 전기변색 효율을 크게 향상시킬 수 있는 이산화티탄-염화리튬 전기변색물질을 제공하는데 있다.']
['전기 화학 디바이스 및 그것을 형성하는 방법', '전기 화학 디바이스에 대해서, 개시한다.전기 화학 디바이스는, 제 1의 투명 도전층, 제 1의 투명 도전층 위에 있는 에렉트로 크로믹층, 에렉트로 크로믹층 위에 있는 대향 전극층, 제 2의 투명 도전층을 갖추어 23°C으로 0.68초/mm이하의 스위칭 속도 파라미터를 가진다.', '제 1의 투명 도전층과 전기 제 1

In [13]:
# 문장 클린징 처리 전
# ['이산화티탄-염화리튬 전기변색물질, 이를 이용한전기변색장치용 파우더 및 필름전극의 제조방법', '본 발명은 이산화티탄-염화리튬 전기변색물질 및 이를 이용한 전기변색장치용 파우더 및 필름전극의 제조방법에 관한 것으로, 이산화티탄에 염화리튬이 균일하게 혼합된 물질을 합성하여 제조되며 상기 이산화티탄-염화리튬에서 티탄 대 리튬의 몰 함량 비율이 1 : x (0.5 &lt; x &lt; 2) 범위를 갖는 이산화티탄-염화리튬을 졸-겔 방법으로 합성함으로써 표면적이 극대화되는 다공성화 또는 그 물질 내에 존재하는 높은 함량의 리튬 이온으로 인하여 전기변색 감응시간을 획기적으로 줄이고 전기변색 효율을 크게 향상시킬 수 있는 매우 뛰어난 발명인 것이다.', '이산화티탄-염화리튬 파우더에 아세틸아세톤이 포함된 증류수를 첨가하여 끈적거리는 상태가 될 때까지 교반하는 단계; 상기 용액에 증류수와 트리톤 X-100을 첨가하여 완전히 균일한 용액이 될 때까지 다시 교반하는 단계; 및 상기 용액을 전도성 유리인 ITO 위에 얇게 입힌 후, 상온에서 건조한 다음 열처리하는 단계를 포함함을 특징으로 하는 전기변색장치용 이산화티탄-염화리튬 필름전극의 제조방법.', '1@@발명은 상기의 제문제점을 감안하여 안출한 것으로, 본 발명에서는 이산화티탄-염화리튬을 새롭게 합성하여 전기변색 물질로서 실용화함에 그 목적이 있다.', '2@@발명의 다른 목적은 전기변색 감응시간을 획기적으로 줄이고 전기변색 효율을 크게 향상시킬 수 있는 이산화티탄-염화리튬 전기변색물질을 제공하는데 있다.']
# ['사모 크로믹성을 가지는 이산화 바나듐 함유 입자이며,', '산소 및 바나듐 이외의 원소로서, 폴링의 전기 음성도가 1.', '６５〜２． 05의 범위내인 원소 A와 폴링의 전기 음성도가 2.', '１０〜２． 54의 범위내인 원소 B만이 함유 되어', '바나듐(100 atom%)에 대해, 전기 원소 A의 총함유량이 0.5~20 atom%의 범위내이며, 한편, 전기 원소 B의 총함유량이 0.05~20 atom%의 범위내인 이산화 바나듐 함유 입자.']
# 처리 후에는 1문장으로 처리될 데이터가 무려 5문장으로 나눠진다!

# 문장 클린징 처리 후
# ['이산화티탄-염화리튬 전기변색물질, 이를 이용한전기변색장치용 파우더 및 필름전극의 제조방법', '본 발명은 이산화티탄-염화리튬 전기변색물질 및 이를 이용한 전기변색장치용 파우더 및 필름전극의 제조방법에 관한 것으로, 이산화티탄에 염화리튬이 균일하게 혼합된 물질을 합성하여 제조되며 상기 이산화티탄-염화리튬에서 티탄 대 리튬의 몰 함량 비율이 1 : x (0.5 x 2) 범위를 갖는 이산화티탄-염화리튬을 졸-겔 방법으로 합성함으로써 표면적이 극대화되는 다공성화 또는 그 물질 내에 존재하는 높은 함량의 리튬 이온으로 인하여 전기변색 감응시간을 획기적으로 줄이고 전기변색 효율을 크게 향상시킬 수 있는 매우 뛰어난 발명인 것이다.', '이산화티탄-염화리튬 파우더에 아세틸아세톤이 포함된 증류수를 첨가하여 끈적거리는 상태가 될 때까지 교반하는 단계; 상기 용액에 증류수와 트리톤 X-100을 첨가하여 완전히 균일한 용액이 될 때까지 다시 교반하는 단계; 및 상기 용액을 전도성 유리인 ITO 위에 얇게 입힌 후, 상온에서 건조한 다음 열처리하는 단계를 포함함을 특징으로 하는 전기변색장치용 이산화티탄-염화리튬 필름전극의 제조방법.', '1 발명은 상기의 제문제점을 감안하여 안출한 것으로, 본 발명에서는 이산화티탄-염화리튬을 새롭게 합성하여 전기변색 물질로서 실용화함에 그 목적이 있다.', '2 발명의 다른 목적은 전기변색 감응시간을 획기적으로 줄이고 전기변색 효율을 크게 향상시킬 수 있는 이산화티탄-염화리튬 전기변색물질을 제공하는데 있다.']
# ['전기 화학 디바이스 및 그것을 형성하는 방법', '전기 화학 디바이스에 대해서, 개시한다.전기 화학 디바이스는, 제 1의 투명 도전층, 제 1의 투명 도전층 위에 있는 에렉트로 크로믹층, 에렉트로 크로믹층 위에 있는 대향 전극층, 제 2의 투명 도전층을 갖추어 23°C으로 0.68초/mm이하의 스위칭 속도 파라미터를 가진다.', '제 1의 투명 도전층과 전기 제 1의 투명 도전층 위에 있는 음극 전기 화학층과 전기 음극 전기 화학층 위에 있는 양극 전기 화학층과 제 2의 투명 도전층과 (을)를 갖추는 전기 화학 디바이스이며, 전기 전기 화학 디바이스는, 23°C으로 0.68초/mm이하인 제 1의 스위칭 속도 파라미터, -20°C으로 1.0초/mm이하인 제 2의 스위칭 속도 파라미터, 1.5/log(오옴) 미만인 log|Z|의 impedance파라미터, 또는 -20°C으로 8%미만인 착색 투과률 파라미터 중 적어도 1개를 가지는, 전기 화학 디바이스.', '이산화 바나듐 함유 입자, 사모 크로믹 필름 및 이산화 바나듐 함유 입자의 제조 방법', 
#  '사모 크로믹성을 가지는 이산화 바나듐 함유 입자이며, 산소 및 바나듐 이외의 원소로서, 폴링의 전기 음성도가 1.65〜2.05의 범위내인 원소 A와 폴링의 전기 음성도가 2.10〜2.54의 범위내인 원소 B만이 함유 되어 바나듐(100 atom%)에 대해, 전기 원소 A의 총함유량이 0.5~20 atom%의 범위내이며, 한편, 전기 원소 B의 총함유량이 0.05~20 atom%의 범위내인 이산화 바나듐 함유 입자.']

In [8]:
# 문장 클린징 처리 전
# """
# ['사모 크로믹성을 가지는 이산화 바나듐 함유 입자이며, 산소 및 바나듐 이외의 원소로서, 폴링의 전기 음성도가 1.',
#  '６５〜２． 05의 범위내인 원소 A와 폴링의 전기 음성도가 2.',
#  '１０〜２． 54의 범위내인 원소 B만이 함유 되어 바나듐(100 atom%)에 대해, 전기 원소 A의 총함유량이 0.5~20 atom%의 범위내이며, 한편, 전기 원소 B의 총함유량이 0.05~20 atom%의 범위내인 이산화 바나듐 함유 입자.']
#  위와 같이 소수점 앞 뒤로 공백이 존재하는데, 이로 인해 소수점에서 문장 분리가 되버린다. 오류 조치가 필요
#  """
# 아래와 같이 구두점 앞뒤 블랭크를 제거하여 일단 구분
# row[col] = unicodedata.normalize('NFKC', row[col])
# https://blog.naver.com/PostView.nhn?blogId=duswl0319&logNo=221516880642&from=search&redirect=Log&widgetTypeCall=true&directAccess=false
# row[col] = row[col].replace(' . ','.')
# row[col] = row[col].replace(' ． ','.')
# row[col] = row[col].replace('． ','.')
# row[col] = row[col].replace('. ','.')
# row[col] = row[col].replace('  ',' ')

### 학습 데이터 분리

In [10]:
# dataset  # Dataset({features: ['text'], num_rows: 16411})
# d  # DatasetDict({train: Dataset({features: ['text'], num_rows: 14769}) test: Dataset({features: ['text'], num_rows: 1642})})
# d["train"]

In [12]:
# data_train1 = data_pre_process(data_train)
# data_test1 = data_pre_process(data_test)

In [13]:
# # download and prepare cc_news dataset
# dataset = load_dataset("cc_news", split="train")
# dataset[0]
# # split the dataset into training (90%) and testing (10%)
# d = dataset.train_test_split(test_size=0.1)
# d["train"], d["test"]
# for t in d["train"]["text"][:3]:
#     print(t)
#     print("="*50)

### using personal dataset
* https://huggingface.co/docs/datasets/dataset_script

In [14]:
# # if you have huge custom dataset separated into files
# # load the splitted files
# files = ["train1.txt", "train2.txt"] # train3.txt, etc.
# dataset = load_dataset("text", data_files=files, split="train")

### Training the Tokenizer

In [8]:
special_tokens = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "<S>", "<T>"]
# if you want to train the tokenizer on both sets
# files = ["train.txt", "test.txt"]
files = ["data/c09k_corpus.txt"]

In [19]:
# initialize the WordPiece tokenizer
tokenizer = BertWordPieceTokenizer(handle_chinese_chars=False)
# tokenizer = BertWordPieceTokenizer(lowercase=False)  # 이 옵션을 줬더니 나중에 ENCODE 결과가 죄다 unk로 ㅠㅠ
# train the tokenizer
tokenizer.train(files=files, vocab_size=vocab_size, special_tokens=special_tokens, show_progress=True)
# enable truncation up to the maximum 512 tokens
tokenizer.enable_truncation(max_length=max_length)

In [11]:
# model_path = "c09k_pretrained_bert"

In [21]:
# make the directory if not already there
if not os.path.isdir(model_path):
    os.mkdir(model_path)
# save the tokenizer  
tokenizer.save_model(model_path)
tokenizer.save('c09k_pretrained_bert/tokenizer.json')

### Pre-train data 생성

In [9]:
try:
    data_train = Dataset.from_text('data/c09k_pre_train.txt')
    data_test = Dataset.from_text('data/c09k_pre_test.txt')

except:
    dataset = load_corpus()
    d["train"], d["test"] = split_dataset()
    dataset_to_text(d["train"], "data/c09k_pre_train.txt")
    dataset_to_text(d["test"], "data/c09k_pre_test.txt")


In [10]:
# dumping some of the tokenizer config to config file, 
# including special tokens, whether to lower case and the maximum sequence length
with open(os.path.join(model_path, "config.json"), "w") as f:
    tokenizer_cfg = {"do_lower_case": True,
                     "unk_token": "[UNK]",
                     "sep_token": "[SEP]",
                     "pad_token": "[PAD]",
                     "cls_token": "[CLS]",
                     "mask_token": "[MASK]",
                     "model_max_length": max_length,
                     "max_len": max_length,
                    }
    json.dump(tokenizer_cfg, f)

In [11]:
# when the tokenizer is trained and configured, load it as BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained(model_path, vocab_size=8000, local_files_only=True)
# tokenizer = BertTokenizer.from_pretrained(model_path, )


loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file c09k_pretrained_bert/config.json
Model config BertConfig {
  "_name_or_path": "c09k_pretrained_bert",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "cls_token": "[CLS]",
  "do_lower_case": true,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "mask_token": "[MASK]",
  "max_len": 512,
  "max_position_embeddings": 512,
  "model_max_length": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token": "[PAD]",
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "sep_token": "[SEP]",
  "transformers_version": "4.22.0.dev0",
  "type_vocab_size": 2,
  "unk_token": "[UNK]",
  "use_cache": true,
  "vocab_size": 30522
}



vocab_file vocab.txt
tokenizer_file tokenizer.json
added_tokens_file added_tokens.json
special_tokens_map_file special_tokens_map.json
tokenizer_config_file tokenizer_config.json


In [12]:
# tokenizer.vocab_size

### Tokenizing the Dataset

In [13]:
def encode_with_truncation(examples):
    """Mapping function to tokenize the sentences passed with truncation"""
    return tokenizer(examples["text"], truncation=True, padding="max_length",
                                     max_length=max_length, return_special_tokens_mask=True)

def encode_without_truncation(examples):
    """Mapping function to tokenize the sentences passed without truncation"""
    return tokenizer(examples["text"], return_special_tokens_mask=True)

In [14]:
# the encode function will depend on the truncate_longer_samples variable
encode = encode_with_truncation if truncate_longer_samples else encode_without_truncation

In [15]:
# # tokenizing the train dataset
# train_dataset = data_train1['text'].map(encode, batched=True)
# test_dataset = data_test1['text'].map(encode, batched=True)

In [16]:
# d["train"]
# train_dataset['attention_mask'][0]

In [37]:
train_dataset

[{'input_ids': [21, 18, 1399, 1037, 377, 16, 12, 41, 13, 3940, 230, 578, 1201, 1678, 1086, 462, 441, 401, 943, 1564, 984, 1091, 2221, 636, 595, 616, 1006, 18], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'special_tokens_mask': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]},
 {'input_ids': [37, 1075, 1527, 39, 863, 1269, 389, 960, 745, 1803, 605, 595, 616, 551, 5114, 3104, 501, 815, 522, 18], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'special_tokens_mask': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]},
 {'input_ids': [703, 7207, 1497, 1509, 485, 3029, 783, 581, 1705, 16, 1638, 1765, 652, 6935, 4361, 389, 2807, 703, 1756, 1033, 914, 18], 'token_type_ids': [

In [30]:
encode(data_train[0])

{'input_ids': [21, 18, 1399, 1037, 377, 16, 12, 41, 13, 3940, 230, 578, 1201, 1678, 1086, 462, 441, 401, 943, 1564, 984, 1091, 2221, 636, 595, 616, 1006, 18], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'special_tokens_mask': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [33]:
train_dataset = list(map(lambda x: encode(x), data_train))
test_dataset = list(map(lambda x: encode(x), data_test))

In [32]:
# train_dataset

[{'input_ids': [21, 18, 1399, 1037, 377, 16, 12, 41, 13, 3940, 230, 578, 1201, 1678, 1086, 462, 441, 401, 943, 1564, 984, 1091, 2221, 636, 595, 616, 1006, 18], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'special_tokens_mask': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]},
 {'input_ids': [37, 1075, 1527, 39, 863, 1269, 389, 960, 745, 1803, 605, 595, 616, 551, 5114, 3104, 501, 815, 522, 18], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'special_tokens_mask': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]},
 {'input_ids': [703, 7207, 1497, 1509, 485, 3029, 783, 581, 1705, 16, 1638, 1765, 652, 6935, 4361, 389, 2807, 703, 1756, 1033, 914, 18], 'token_type_ids': [

In [17]:
# tokenizing the train dataset
train_dataset = d["train"].map(encode, batched=True)
# tokenizing the testing dataset
test_dataset = d["test"].map(encode, batched=True)

NameError: name 'd' is not defined

In [34]:
# print(train_dataset[:2])
len(train_dataset)  # 14769

14769

In [35]:
if truncate_longer_samples:
    # remove other columns and set input_ids and attention_mask as PyTorch tensors
    train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
    test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
else:
    # remove other columns, and remain them as Python lists
    test_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])
    train_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])

AttributeError: 'list' object has no attribute 'set_format'

In [33]:
from itertools import chain
# Main data processing function that will concatenate all texts from our dataset and generate chunks of max_seq_length.
# grabbed from: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py

In [34]:
# # train_dataset.features.keys()
# # dict_keys(['text', 'input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'])
# total_length = len(list(chain(train_dataset['text'])))  # total_length = 14769
# max_length  # 512
# total_length = (total_length // max_length) * max_length
# # total_length  # 14336, total_length // max_length = 28.8457..., 28 * 512 = 14336

In [35]:
max_length = 64

In [36]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= max_length:
        total_length = (total_length // max_length) * max_length
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + max_length] for i in range(0, total_length, max_length)]
        for k, t in concatenated_examples.items()
    }
    return result

In [37]:
# !pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
# https://pytorch.org/get-started/locally/#windows-package-manager

In [38]:
# Note that with `batched=True`, this map processes 1,000 texts together, so group_texts throws away a
# remainder for each of those groups of 1,000 texts. You can adjust that batch_size here but a higher value
# might be slower to preprocess.
#
# To speed up this part, we use multiprocessing. See the documentation of the map method for more information:
# https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map
if not truncate_longer_samples:
    train_dataset1 = train_dataset.map(group_texts, batched=True,
                                                                        desc=f"Grouping texts in chunks of {max_length}")
    test_dataset1 = test_dataset.map(group_texts, batched=True,
                                                                    desc=f"Grouping texts in chunks of {max_length}")
    # convert them from lists to torch tensors
    train_dataset1.set_format("torch")
    test_dataset1.set_format("torch")

Grouping texts in chunks of 64: 100%|██████████| 2/2 [00:00<00:00, 15.49ba/s]


In [39]:
# len(train_dataset), len(test_dataset)  # (14769, 1642)
len(train_dataset1), len(test_dataset1)  # (2171, 225)

(19400, 2047)

In [41]:
for i in range(2):
    print(train_dataset1['input_ids'][i])
    print(tokenizer.decode(train_dataset1['input_ids'][i]))

tensor([3418, 3222, 3743,  603,  648,  824,  517,   16, 1914,  824,   16, 2966,
          17,  413,  851, 4105, 4721,  824,   16, 2175,  824,   16,  994,  824,
          16, 3985,  824,   16,  994,   17, 2181,  824,   16, 7448, 7696,  824,
          16, 7448, 7696,   17,  413,  851, 4105, 4721,  824,   16,  701,  824,
         412, 1394, 6904,  731, 3524, 4246, 4809, 2510, 3765,  920, 3947, 2556,
        2351,   12,  820,   13])
소수성기와 친수성기를 포함한 수지이며, 에폭시 수지, 페놀 - 포름알데히드 수지, 폴리우레탄 수지, 아크릴 수지, 폴리에스텔 수지, 아크릴 - 우레탄 수지, 멜라민 수지, 멜라민 - 포름알데히드 수지, 아미노 수지 및 이러한 편성으로부터 선택되는 수지와 물과 임의에 염화 성분과 이온 강도 조정제와 ( 을 )
tensor([ 797,  656,  573,   16, 2152, 1493, 1025,  656, 5059, 1542,   16,  862,
         230, 5017, 4945, 1137, 4789,  824, 5537,  481,   16, 5814, 3834,   16,
          20, 1118,   70,   32, 1157, 3188,  730, 5273,   16, 5277, 1121, 2013,

### Loading the Model

In [42]:
vocab_size

8000

In [43]:
# initialize the model with the config
model_config = BertConfig(vocab_size=vocab_size, max_position_embeddings=max_length)
model = BertForMaskedLM(config=model_config)

In [48]:
# model_config.vocab_size

### Training

In [49]:
# initialize the data collator, randomly masking 20% (default is 15%) of the tokens for the Masked Language
# Modeling (MLM) task
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [50]:
training_args = TrainingArguments(
    output_dir=model_path,          # output directory to where save model checkpoint
    evaluation_strategy="steps",    # 'steps': evaluate each `logging_steps`, 'epoch'  : each epoch
    overwrite_output_dir=True,      
    num_train_epochs=50.,            # number of training epochs, feel free to tweak
    per_device_train_batch_size=32, # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=8,  # accumulating the gradients before updating the weights
    per_device_eval_batch_size=8,  # evaluation batch size
    logging_steps=100,             # evaluate, log and save model checkpoints every 1000 step
    save_steps=1000,
    load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
    # save_total_limit=3,           # whether you don't have much space so you let only 3 model weights saved in the disk
)

using `logging_steps` to initialize `eval_steps` to 100
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [51]:
model_path

'c09k_pretrained_bert'

In [52]:
# initialize the trainer and pass everything to it
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset1,
    eval_dataset=test_dataset1,
)

In [53]:
# train the model
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
/home/hdh/PycharmProjects/bert_pretrain/venv/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 19400
  Num Epochs = 50
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 8
  Total optimization steps = 3750


Step,Training Loss,Validation Loss
100,7.295600,6.732242
200,6.603300,6.342851
300,6.378600,6.237882
400,6.275000,6.045237
500,6.102300,5.956049
600,6.024500,5.918322
700,5.994600,5.867238
800,5.883700,5.784225
900,5.825500,5.757144
1000,5.819400,5.694105


The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2047
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2047
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2047
  Batch size = 8
The follow

***** Running Evaluation *****
  Num examples = 2047
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2047
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2047
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Runn

TrainOutput(global_step=3750, training_loss=5.451891813151041, metrics={'train_runtime': 6405.2907, 'train_samples_per_second': 151.437, 'train_steps_per_second': 0.585, 'total_flos': 3.18986161668096e+16, 'train_loss': 5.451891813151041, 'epoch': 49.99})

In [56]:
model.save_pretrained(os.path.join(model_path, 'checkpoint-4320'))

Configuration saved in c09k_pretrained_bert/checkpoint-4320/config.json
Model weights saved in c09k_pretrained_bert/checkpoint-4320/pytorch_model.bin


In [ ]:
# Gradient Accumulation steps * Total optimization steps = 8 * 540 = 4320

In [ ]:
# model.save_pretrained(model_path)

### Additional Training the Model

In [74]:
# load the model checkpoint
model1 = BertForMaskedLM.from_pretrained(os.path.join(model_path, "checkpoint-4320"))
# load the tokenizer
# tokenizer = BertTokenizerFast.from_pretrained(model_path)
tokenizer1 = BertTokenizerFast.from_pretrained(model_path, vocab_size=8000, local_files_only=True)

loading configuration file c09k_pretrained_bert/checkpoint-4320/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 64,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.22.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 8000
}

loading weights file c09k_pretrained_bert/checkpoint-4320/pytorch_model.bin
All model checkpoint weights were used when initializing BertForMaskedLM.

All the weights of BertForMaskedLM were initialized from the model checkpoint at c09k_pretrained_bert/checkpoint-4320.
If your task is similar to the task t

vocab_file vocab.txt
tokenizer_file tokenizer.json
added_tokens_file added_tokens.json
special_tokens_map_file special_tokens_map.json
tokenizer_config_file tokenizer_config.json


In [75]:
fill_mask = pipeline("fill-mask", model=model1, tokenizer=tokenizer1)

In [76]:
# perform predictions
# 인광성 유기 금속 이리듐 착체, 발광 소자, 발광 장치, 전자 기기, 및 조명 장치
# 본 명세서는 화학식 1로 표시되는 화합물 및 이를 포함하는 유기 발광 소자에 관한 것이다
examples = [
    "인광성 유기 금속 이리듐 착체, [MASK] 소자, 발광 장치, 전자 기기, 및 조명 장치",
    "본 명세서는 화학식 1로 표시되는 [MASK] 및 이를 포함하는 유기 발광 소자에 관한 것이다",
    "인광성 유기 금속 이리듐 착체, 발광 소자, 발광 장치, [MASK] 기기, 및 조명 장치",
    "본 명세서는 화학식 1로 표시되는 화합물 및 이를 포함하는 [MASK] 발광 소자에 관한 것이다",
]
for example in examples:
    print(fill_mask(example))
    for prediction in fill_mask(example):
        print(f"{prediction['sequence']}, confidence: {prediction['score']}")
    print("="*50)

[{'score': 0.045564088970422745, 'token': 678, 'token_str': '유기', 'sequence': '인광성 유기 금속 이리듐 착체, 유기 소자, 발광 장치, 전자 기기, 및 조명 장치'}, {'score': 0.04470434784889221, 'token': 16, 'token_str': ',', 'sequence': '인광성 유기 금속 이리듐 착체,, 소자, 발광 장치, 전자 기기, 및 조명 장치'}, {'score': 0.027636609971523285, 'token': 562, 'token_str': '재료', 'sequence': '인광성 유기 금속 이리듐 착체, 재료 소자, 발광 장치, 전자 기기, 및 조명 장치'}, {'score': 0.023211440071463585, 'token': 450, 'token_str': '전기', 'sequence': '인광성 유기 금속 이리듐 착체, 전기 소자, 발광 장치, 전자 기기, 및 조명 장치'}, {'score': 0.017397301271557808, 'token': 18, 'token_str': '.', 'sequence': '인광성 유기 금속 이리듐 착체,. 소자, 발광 장치, 전자 기기, 및 조명 장치'}]
인광성 유기 금속 이리듐 착체, 유기 소자, 발광 장치, 전자 기기, 및 조명 장치, confidence: 0.045564088970422745
인광성 유기 금속 이ᄅ

In [78]:
# Num examples = 19400
# Num Epochs = 50
# Total optimization steps = 3750 = 750*50 = 
# 로드한 모델 추가학습
training_args = TrainingArguments(
    output_dir=model_path,          # output directory to where save model checkpoint
    evaluation_strategy="steps",    # 'steps': evaluate each `logging_steps`, 'epoch'  : each epoch
    overwrite_output_dir=True,      
    num_train_epochs=100.,            # number of training epochs, feel free to tweak
    per_device_train_batch_size=32, # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=8,  # accumulating the gradients before updating the weights
    per_device_eval_batch_size=8,  # evaluation batch size
    logging_steps=1500,             # evaluate, log and save model checkpoints every 1000 step
    save_steps=1500,
    load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
    save_total_limit=5,           # whether you don't have much space so you let only 3 model weights saved in the disk
)

trainer = Trainer(
    model=model1,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset1,
    eval_dataset=test_dataset1,
)

trainer.train()

using `logging_steps` to initialize `eval_steps` to 1500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 19400
  Num Epochs = 100
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 8
  Total optimization steps = 7500


Step,Training Loss,Validation Loss
1500,4.143100,3.477852
3000,3.073100,2.746345
4500,2.481400,2.393077
6000,2.170100,2.215955
7500,2.018300,2.143648


The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2047
  Batch size = 8
Saving model checkpoint to c09k_pretrained_bert/checkpoint-1500
Configuration saved in c09k_pretrained_bert/checkpoint-1500/config.json
Model weights saved in c09k_pretrained_bert/checkpoint-1500/pytorch_model.bin
Deleting older checkpoint [c09k_pretrained_bert/checkpoint-2000] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2047
  Batch size = 8
Saving model checkpoint to c09k_

TrainOutput(global_step=7500, training_loss=2.7772110026041665, metrics={'train_runtime': 12492.1984, 'train_samples_per_second': 155.297, 'train_steps_per_second': 0.6, 'total_flos': 6.38038107242496e+16, 'train_loss': 2.7772110026041665, 'epoch': 99.99})

In [80]:
# load the model checkpoint
model1 = BertForMaskedLM.from_pretrained(os.path.join(model_path, "checkpoint-7500"))
# load the tokenizer
# tokenizer = BertTokenizerFast.from_pretrained(model_path)
tokenizer1 = BertTokenizerFast.from_pretrained(model_path, vocab_size=8000, local_files_only=True)
fill_mask = pipeline("fill-mask", model=model1, tokenizer=tokenizer1)
# perform predictions
# 인광성 유기 금속 이리듐 착체, 발광 소자, 발광 장치, 전자 기기, 및 조명 장치
# 본 명세서는 화학식 1로 표시되는 화합물 및 이를 포함하는 유기 발광 소자에 관한 것이다
examples = [
    "인광성 유기 금속 이리듐 착체, [MASK] 소자, 발광 장치, 전자 기기, 및 조명 장치",
    "본 명세서는 화학식 1로 표시되는 [MASK] 및 이를 포함하는 유기 발광 소자에 관한 것이다",
    "인광성 유기 금속 이리듐 착체, 발광 소자, 발광 장치, [MASK] 기기, 및 조명 장치",
    "본 명세서는 화학식 1로 표시되는 화합물 및 이를 포함하는 [MASK] 발광 소자에 관한 것이다",
]
for example in examples:
    print(fill_mask(example))
    for prediction in fill_mask(example):
        print(f"{prediction['sequence']}, confidence: {prediction['score']}")
    print("="*50)

loading configuration file c09k_pretrained_bert/checkpoint-7500/config.json
Model config BertConfig {
  "_name_or_path": "c09k_pretrained_bert/checkpoint-4320",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 64,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.22.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 8000
}

loading weights file c09k_pretrained_bert/checkpoint-7500/pytorch_model.bin
All model checkpoint weights were used when initializing BertForMaskedLM.

All the weights of BertForMaskedLM were initialized from the model checkpoint at c09k_pretrained_

vocab_file vocab.txt
tokenizer_file tokenizer.json
added_tokens_file added_tokens.json
special_tokens_map_file special_tokens_map.json
tokenizer_config_file tokenizer_config.json
[{'score': 0.46306195855140686, 'token': 1193, 'token_str': '발광', 'sequence': '인광성 유기 금속 이리듐 착체, 발광 소자, 발광 장치, 전자 기기, 및 조명 장치'}, {'score': 0.13777363300323486, 'token': 737, 'token_str': '표시', 'sequence': '인광성 유기 금속 이리듐 착체, 표시 소자, 발광 장치, 전자 기기, 및 조명 장치'}, {'score': 0.05654741823673248, 'token': 1125, 'token_str': '일렉트로크로믹', 'sequence': '인광성 유기 금속 이리듐 착체, 일렉트로크로믹 소자, 발광 장치, 전자 기기, 및 조명 장치'}, {'score': 0.03776992857456207, 'token': 3316, 'token_str': '광기능', 'sequence': '인광성 유기 금속 이리듐 착체, 광기능 소자, 발광 장치, 전자 기기, 및 조명 장치'}, {'score': 0.03028818406164646, 'token': 663, 'token_str': '전기변색', 'sequence': '인과

### Etc

In [28]:
# dir(tokenizers.Tokenizer)
# ?BertWordPieceTokenizer
# BertWordPieceTokenizer(
#     vocab: Union[str, Dict[str, int], NoneType] = None,
#     unk_token: Union[str, tokenizers.AddedToken] = '[UNK]',
#     sep_token: Union[str, tokenizers.AddedToken] = '[SEP]',
#     cls_token: Union[str, tokenizers.AddedToken] = '[CLS]',
#     pad_token: Union[str, tokenizers.AddedToken] = '[PAD]',
#     mask_token: Union[str, tokenizers.AddedToken] = '[MASK]',
#     clean_text: bool = True,
#     handle_chinese_chars: bool = True,
#     strip_accents: Union[bool, NoneType] = None,
#     lowercase: bool = True,
#     wordpieces_prefix: str = '##',
# )
# Docstring:      Bert WordPiece Tokenizer 
# File:           ~/PycharmProjects/bert_pretrain/venv/lib/python3.8/site-packages/tokenizers/implementations/bert_wordpiece.py
# Type:           type
# Subclasses:     

In [29]:
# ?tokenizer
# Union은 공용체 형으로 Union[X, Y]는 X나 Y를 의미
# https://python.flowdas.com/library/typing.html#typing.Union
    
# Signature:     
# tokenizer(
#     text: Union[str, List[str], List[List[str]]] = None,
#     text_pair: Union[str, List[str], List[List[str]], NoneType] = None,
#     text_target: Union[str, List[str], List[List[str]]] = None,
#     text_pair_target: Union[str, List[str], List[List[str]], NoneType] = None,
#     add_special_tokens: bool = True,
#     padding: Union[bool, str, transformers.utils.generic.PaddingStrategy] = False,
#     truncation: Union[bool, str, transformers.tokenization_utils_base.TruncationStrategy] = False,
#     max_length: Union[int, NoneType] = None,
#     stride: int = 0,
#     is_split_into_words: bool = False,
#     pad_to_multiple_of: Union[int, NoneType] = None,
#     return_tensors: Union[str, transformers.utils.generic.TensorType, NoneType] = None,
#     return_token_type_ids: Union[bool, NoneType] = None,
#     return_attention_mask: Union[bool, NoneType] = None,
#     return_overflowing_tokens: bool = False,
#     return_special_tokens_mask: bool = False,
#     return_offsets_mapping: bool = False,
#     return_length: bool = False,
#     verbose: bool = True,
#     **kwargs,
# ) -> transformers.tokenization_utils_base.BatchEncoding

In [30]:
# # 과거 데이터 처리(위의 여러 파일 처리로 대체 08.19)
# data = pd.read_excel('data/c09k_biblio(861건).xlsx')
# data = data[['발행번호', '발명의명칭', '요약', '대표청구항', '과제', '해결방안']]
# # data_train, data_test = train_test_split(data, test_size=0.1, random_state=15, shuffle=True)
# # print(data_train.shape, data_test.shape)
# def data_pre_process(data):
#     data['text'] = data[['발명의명칭', '요약', '대표청구항', '과제', '해결방안']].apply(". ".join, axis=1)
#     data['text'] = data['text'].str.replace('\n\t',' ')
#     data['text'] = data['text'].str.replace('\t\t',' ')
#     data['text'] = data['text'].str.replace('.  .','.')
#     data['text'] = data['text'].str.replace('. .','.')
#     data['text'] = data['text'].str.replace('..','.')
#     data['text'] = data['text'].str.replace('  ',' ')
#     data['text'] = data['text'].str.strip()
#     data1 = data[['발행번호', 'text']].copy()
# #     data1.set_index('발행번호', inplace=True)
#     return data1
# data_text = data_pre_process(data)
# dataset = Dataset.from_pandas(data_text)

In [31]:
# # 김명선 책임의 문장 분리 코드(미사용)
# REG_SENT_KO=r'([ㄱ-ㅣ가-힣]+[.]|[\n]|[:;!?])'
# REG_SENT_EN=r'([a-zA-Z]+[.]\s|[\n]|[:;!?])'

# def split_sentence(doc, regex):
#     s = 0
#     for m in re.finditer(regex, doc):
#         sent = doc[s:m.end()].strip()
#         s = m.end()
#         if not sent:
#             continue
#         yield sent

#     if s < len(doc):
#         sent = doc[s:].strip()
#         if sent:
#             yield sent

In [33]:
# # initialize the WordPiece tokenizer
# # tokenizer = BertWordPieceTokenizer()
# tokenizer = BertWordPieceTokenizer(handle_chinese_chars=False)
# # tokenizer = BertWordPieceTokenizer(lowercase=False)  # 이 옵션을 줬더니 나중에 ENCODE 결과가 죄다 unk로 ㅠㅠ
# # tokenizer = BertWordPieceTokenizer(handle_chinese_chars=False, lowercase=False)  # 이 옵션을 줬더니 나중에 ENCODE 결과가 죄다 unk로 ㅠㅠ
# # train the tokenizer
# tokenizer.train(files=files, vocab_size=vocab_size, special_tokens=special_tokens, show_progress=True)
# # enable truncation up to the maximum 512 tokens
# tokenizer.enable_truncation(max_length=max_length)

In [45]:
# def sent_tokenize(input='./input.txt', output='./output.txt'):
#     sent_tokenizer = load("tokenizers/punkt/english.pickle")
#     extra_abbreviations = [
#         'RE','re','pat', 'no', 'nos','vol','jan','feb','mar','apr','jun',
#         'jul','aug','sep','oct','nov','dec','eng','ser','ind','ed','pp',
#         'e.g','al','T.E.N.S', 'E.M.S','F.E','U.H.T.S.T','degree',
#         '/gm','A','B','C','D','E','F','G','H','I','J','K','L','M','N','O',
#         'P','Q','R','S','T','U','V','W','X','Y','Z']
#     sent_tokenizer._params.abbrev_types.update(extra_abbreviations)

#     load_file=open(input,'r')
#     save_file=open(output,'w')
#     no_blank = False
#     while True:
#         line = load_file.readline()
#         if line == "":
#             break
#         if line.strip() == "":
#             if no_blank:
#                 continue
#             save_file.write(f"{line}")
#         else:
#             print(line)
#             result_ = tokenizer.tokenize(line)
#             print(result_)
#             result  = [ f"{cur_line}\n" for cur_line in result_ ]
#             for save_line in result:
#                 save_file.write(save_line)